In [1]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [ ]:
import os
from google import genai

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

client = genai.Client(api_key=GOOGLE_API_KEY)

In [10]:
for m in client.models.list():
    print(m.name, m.supported_actions)

models/embedding-gecko-001 ['embedText', 'countTextTokens']
models/gemini-1.5-pro-latest ['generateContent', 'countTokens']
models/gemini-1.5-pro-002 ['generateContent', 'countTokens', 'createCachedContent']
models/gemini-1.5-pro ['generateContent', 'countTokens']
models/gemini-1.5-flash-latest ['generateContent', 'countTokens']
models/gemini-1.5-flash ['generateContent', 'countTokens']
models/gemini-1.5-flash-002 ['generateContent', 'countTokens', 'createCachedContent']
models/gemini-1.5-flash-8b ['createCachedContent', 'generateContent', 'countTokens']
models/gemini-1.5-flash-8b-001 ['createCachedContent', 'generateContent', 'countTokens']
models/gemini-1.5-flash-8b-latest ['createCachedContent', 'generateContent', 'countTokens']
models/gemini-2.5-pro-preview-03-25 ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.5-flash-preview-05-20 ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.5-fl

In [11]:
for m in client.models.list():
    if "embedContent" in m.supported_actions:
        print(m.name)

models/embedding-001
models/text-embedding-004
models/gemini-embedding-exp-03-07
models/gemini-embedding-exp
models/gemini-embedding-001


In [ ]:
# PDF/csvs/docx/doc/webpages/ > ExtractContent -> TextSplitters -> Chunking -> Embedding -> VectorDB


# Query: user_question -> embeddings -> based on similarity -> top n most similar documents




In [12]:
DOCUMENT1 = """
  Operating the Climate Control System  Your Googlecar has a climate control
  system that allows you to adjust the temperature and airflow in the car.
  To operate the climate control system, use the buttons and knobs located on
  the center console.  Temperature: The temperature knob controls the
  temperature inside the car. Turn the knob clockwise to increase the
  temperature or counterclockwise to decrease the temperature.
  Airflow: The airflow knob controls the amount of airflow inside the car.
  Turn the knob clockwise to increase the airflow or counterclockwise to
  decrease the airflow. Fan speed: The fan speed knob controls the speed
  of the fan. Turn the knob clockwise to increase the fan speed or
  counterclockwise to decrease the fan speed.
  Mode: The mode button allows you to select the desired mode. The available
  modes are: Auto: The car will automatically adjust the temperature and
  airflow to maintain a comfortable level.
  Cool: The car will blow cool air into the car.
  Heat: The car will blow warm air into the car.
  Defrost: The car will blow warm air onto the windshield to defrost it.
"""
DOCUMENT2 = """
  Your Googlecar has a large touchscreen display that provides access to a
  variety of features, including navigation, entertainment, and climate
  control. To use the touchscreen display, simply touch the desired icon.
  For example, you can touch the \"Navigation\" icon to get directions to
  your destination or touch the \"Music\" icon to play your favorite songs.
"""
DOCUMENT3 = """
  Shifting Gears Your Googlecar has an automatic transmission. To
  shift gears, simply move the shift lever to the desired position.
  Park: This position is used when you are parked. The wheels are locked
  and the car cannot move.
  Reverse: This position is used to back up.
  Neutral: This position is used when you are stopped at a light or in traffic.
  The car is not in gear and will not move unless you press the gas pedal.
  Drive: This position is used to drive forward.
  Low: This position is used for driving in snow or other slippery conditions.
"""

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [ ]:
import chromadb.utils.embedding_functions as embedding_functions
google_ef  = embedding_functions.GoogleGenerativeAiEmbeddingFunction(GOOGLE_API_KEY)



In [34]:
from chromadb.utils import embedding_functions
EMBED_MODEL = 'all-MiniLM-L6-v2'

embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name=EMBED_MODEL
)


In [56]:
import chromadb

chroma_client = chromadb.PersistentClient(path=".chroma_demo")

In [57]:
collection_name = "google_car_docs2"

db = chroma_client.get_or_create_collection(
    name=collection_name,
    embedding_function=embedding_func

)


In [29]:


for i, d in enumerate(documents, start=1):
    print(i, d[:20])

1 
  Operating the Cli
2 
  Your Googlecar ha
3 
  Shifting Gears Yo


In [ ]:
# metadata - id
# documents

for i, d in enumerate(documents, start=1):

    db.add(
        documents=d,
        ids=str(i)    )



In [47]:
query = 'touch screen features'  #-> embeddings ->

context = db.query(query_texts=[query], n_results=2)['documents'][0][0]

In [48]:
context

'\n  Your Googlecar has a large touchscreen display that provides access to a\n  variety of features, including navigation, entertainment, and climate\n  control. To use the touchscreen display, simply touch the desired icon.\n  For example, you can touch the "Navigation" icon to get directions to\n  your destination or touch the "Music" icon to play your favorite songs.\n'

In [54]:

user_prompt = f"""

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

Question: {query}
Context: {context}
Answer:
"""

user_prompt

'\n\nYou are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don\'t know the answer, just say that you don\'t know. Use three sentences maximum and keep the answer concise.\n\nQuestion: touch screen features\nContext: \n  Your Googlecar has a large touchscreen display that provides access to a\n  variety of features, including navigation, entertainment, and climate\n  control. To use the touchscreen display, simply touch the desired icon.\n  For example, you can touch the "Navigation" icon to get directions to\n  your destination or touch the "Music" icon to play your favorite songs.\n\nAnswer:\n'

In [ ]:
from google import genai


geemini_client = genai.Client(api_key=os.getenv('GOOGLE_API_KEY'))

In [ ]:

response = geemini_client.models.generate_content(
    model="gemini-2.5-flash",
    contents=user_prompt
)
print(response.text)

The Googlecar's large touchscreen display provides access to a variety of features, including navigation, entertainment, and climate control. Users can simply touch icons like "Navigation" to get directions or "Music" to play songs.


# CRUD

In [ ]:
collection_name = "google_car_docs2"

db = chroma_client.get_or_create_collection(
    name=collection_name,
    embedding_function=embedding_func

)

db.get(ids='1', include=['documents', 'embeddings'])


{'ids': ['1'],
 'embeddings': array([[-1.27526848e-02,  1.80826318e-02,  2.99500879e-02,
          2.09969543e-02, -2.42913496e-02, -2.52780467e-02,
         -1.79524794e-02, -3.34319733e-02,  9.64219682e-03,
          2.93083638e-02,  7.80505165e-02, -2.86587942e-02,
          1.79007780e-02, -8.96425918e-02,  3.91565226e-02,
          9.25600063e-03,  1.24521498e-02,  2.53139939e-02,
         -6.84027597e-02, -1.05424616e-02,  4.12376635e-02,
         -4.48163450e-02,  4.26927768e-02,  3.85458171e-02,
         -1.01623356e-01, -3.74629796e-02,  1.52260568e-02,
          2.37987358e-02, -3.58580016e-02,  2.66630668e-02,
         -3.27366740e-02,  1.27618713e-02,  3.32325213e-02,
          3.40394266e-02, -1.56909660e-01, -1.50346681e-01,
          2.07297448e-02,  1.40293064e-02, -6.74147010e-02,
          3.96996178e-02, -4.04897854e-02, -9.13927108e-02,
          8.38723406e-03,  1.17111520e-03,  1.51247894e-02,
          9.10741836e-02, -6.10320866e-02,  2.81820260e-02,
          2